# **[KCYPS 2018] 데이터 불러오기**

# **Settings**

In [ ]:
# system
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/메타버스 아카데미/메타버스아카데미 6월 프로젝트/data/KCYPS2018')

import warnings
warnings.filterwarnings('ignore')                                    # warning 출력 false

# dataframe
import re
import pickle
import numpy as np
import pandas as pd

## **1) 데이터 압축 풀기**

In [ ]:
# 압축 풀기
# ! mkdir KCYPS2018-e4
# ! unzip KCYPS2018-e4.zip -d ./KCYPS2018-e4

In [ ]:
# 압축 풀기
# ! mkdir KCYPS2018-m1
# ! unzip KCYPS2018-m1.zip -d ./KCYPS2018-m1

## **2) 데이터 불러오기**

### **변수 설명 데이터**

In [ ]:
# 패널 데이터의 변수 정보를 불러오는 함수이다.
def load_codebooks(file_path):
    codebooks = {'e4':'', 'm1':'', 'p':''}
    sheet_names = ['LAYOUT_e', 'LAYOUT_m', 'LAYOUT_p']

    for dic_name, sheet in zip(codebooks.keys(), sheet_names):
        temp = pd.read_excel(file_path, sheet_name=sheet,
                             skiprows=3, usecols=[i for i in range(1,8)]).reset_index(drop=True)
        temp = temp.iloc[1:,:]
        codebooks[dic_name] = temp

    with open('./datasets/codebooks.pkl', 'wb') as f:
        pickle.dump(codebooks, f)

    print(f'>>> codebooks 저장 완료! -> list={list(codebooks.keys())}')

    return codebooks

In [ ]:
# codebook 불러오기
file_path = 'KCYPS2018 Codebook.xlsx'
codebooks = load_codebooks(file_path)

>>> codebooks 저장 완료! -> list=['e4', 'm1', 'p']


In [ ]:
codebooks['e4'].head(5)

,대영역,중영역,소영역,조사항목,하위요인,변수설명,변수명
1,-,-,-,-,-,가구 ID,HID
2,-,-,-,-,-,개인 ID,PID
3,-,-,-,-,-,학교코드,SCLID
4,-,-,-,-,-,온라인 조사 여부,YONLINE
5,-,-,-,-,-,횡단면_원 가중치,WEIGHTA1


### **원 데이터 불러오기**

In [ ]:
# 패널 데이터를 불러오는 함수이다.
def load_data(codebooks):
    data = {}
    print('info = {e4: 초등4, m1: 중등1, p: 보호자}')
    print('-' * 60)
    for survey in ['e4', 'm1']:
        for year in range(1, 5):
            file_path = f'./KCYPS2018-{survey}/'

            # 각 파일 불러오기
            try:
                data1 = pd.read_csv(file_path + f'KCYPS2018{survey}Yw{year}.csv', encoding='CP949')
                data2 = pd.read_csv(file_path + f'KCYPS2018{survey}Pw{year}.csv', encoding='CP949')
            except:
                data1 = pd.read_csv(file_path + f'KCYPS2018{survey}Yw{year}.csv')
                data2 = pd.read_csv(file_path + f'KCYPS2018{survey}Pw{year}.csv')
            print(f'data Y size = {data1.shape} data P size = {data2.shape}')

            # 데이터 타입 변경
            data1 = data1.astype({'HID':str, 'PID':str})
            data2 = data2.astype({'HID':str, 'PID':str})

            # ID 변수 생성
            data1['ID'] = data1['HID'] + '-' + data1['PID'] + '-' + str(year)
            data2['ID'] = data2['HID'] + '-' + data2['PID'] + '-' + str(year)

            # ID 변수 생성
            data1['ID_w'] = data1['HID'] + '-' + data1['PID']
            data2['ID_w'] = data2['HID'] + '-' + data2['PID']

            # 데이터 변수에 시계열 표현 제거 w1, w2, w3, w4
            data_cols1, data_cols2 = [], []
            for col1 in data1.columns:
                if col1[-2] == 'w':
                    col1 = col1[:-2]
                data_cols1.append(col1)

            for col2 in data2.columns:
                if col2[-2] == 'w':
                    col2 = col2[:-2]
                data_cols2.append(col2)

            # 데이터 변수 변경
            data1.columns = data_cols1
            data2.columns = data_cols2

            data[f'{survey}Yw{year}'] = data1
            data[f'{survey}Pw{year}'] = data2
            print(f'>>> {survey}_{year}차 조사 데이터 불러오기 완료')
            print('-' * 60)

    # 저장
    with open(f'./datasets/data_dict.pkl', 'wb') as f:
        pickle.dump(data, f)

    print('>>> data 저장 완료!')
    print(f'-> list={list(data.keys())}')

    return data

In [ ]:
# 데이터 불러오기
data = load_data(codebooks)

info = {e4: 초등4, m1: 중등1, p: 보호자}
------------------------------------------------------------
data Y size = (2607, 239) data P size = (2607, 209)
>>> e4_1차 조사 데이터 불러오기 완료
------------------------------------------------------------
data Y size = (2607, 266) data P size = (3518, 196)
>>> e4_2차 조사 데이터 불러오기 완료
------------------------------------------------------------
data Y size = (2607, 315) data P size = (3597, 249)
>>> e4_3차 조사 데이터 불러오기 완료
------------------------------------------------------------
data Y size = (2607, 432) data P size = (3552, 241)
>>> e4_4차 조사 데이터 불러오기 완료
------------------------------------------------------------
data Y size = (2590, 350) data P size = (2590, 209)
>>> m1_1차 조사 데이터 불러오기 완료
------------------------------------------------------------
data Y size = (2590, 378) data P size = (3628, 196)
>>> m1_2차 조사 데이터 불러오기 완료
------------------------------------------------------------
data Y size = (2590, 427) data P size = (3505, 249)
>>> m1_3차 조사 데이터 불러오기 완료


In [ ]:
data['e4Pw1'].head(5)

,HID,PID,PARENT,PYBRT1A,PYBRT1B,PYGENDER,PINCOME,PINCOME1,PYBRP1,PYBRP2,...,PMDA1C13,PMDA1C14,PMDA1C15,PPHY1A00,PPHY1B00,PPHY1C00,PPHY2A00,PPHY2B00,ID,ID_w
0,1,1,1,2008,2,2,7.0,3.0,1,1.0,...,2.0,3.0,3.0,3,1,0,155.0,43.0,1-1-1,1-1
1,2,1,1,2008,10,2,5.0,3.0,1,1.0,...,2.0,3.0,2.0,3,1,1,163.0,67.0,2-1-1,2-1
2,3,1,1,2008,6,1,9.0,3.0,1,1.0,...,3.0,3.0,3.0,2,1,7,158.0,60.0,3-1-1,3-1
3,3,2,1,2008,6,1,9.0,3.0,1,1.0,...,3.0,3.0,3.0,2,1,7,158.0,60.0,3-2-1,3-2
4,4,1,1,2008,6,2,5.0,3.0,1,1.0,...,3.0,3.0,2.0,3,5,7,162.0,46.0,4-1-1,4-1


## **2) 컬럼명 정보 추출하기**

### **변수의 설명 추출 딕셔너리**

In [ ]:
# codebook에서 변수의 설명을 딕셔너리 형태로 만드는 함수이다.
# ex. {'HID': '가구 ID','PID': '개인 ID','SCLID': '학교코드','YONLINE': '온라인 조사 여부',...}

def get_variable_info(cookbooks):
    var_dict = {}
    cnt = 0
    for sheet, codebook in codebooks.items():
        var_dict[sheet] = {k:v for k, v in zip(codebook['변수명'],codebook['변수설명'])}

    # 저장
    with open(f'./datasets/variable_info.pkl', 'wb') as f:
        pickle.dump(var_dict, f)

    print('>>> 변수 설명 정보(개별) 저장 완료!')

    return var_dict

In [ ]:
# 변수의 설명 추출 딕셔너리
variable_info = get_variable_info(codebooks)

>>> 변수 설명 정보(개별) 저장 완료!


In [ ]:
# 예시
for x in variable_info.items():
    print(x)

('e4', {'HID': '가구 ID', 'PID': '개인 ID', 'SCLID': '학교코드', 'YONLINE': '온라인 조사 여부', 'WEIGHTA1': '횡단면_원 가중치', 'WEIGHTA2': '횡단면_표준화 가중치', 'WEIGHTB1': '종단면_원 가중치', 'WEIGHTB2': '종단면_표준화 가중치', 'SIBLING': '[형제자매] 원패널과의 관계', 'SIBLING_T': '[형제자매] 원패널과의 관계_기타', 'SGRADE': '[형제자매] 형제자매 학년', 'SURVEY1': '조사 참여 여부_청소년', 'SURVEY2': '조사 참여 여부_보호자', 'COHORT': '코호트 구분', 'ARA1A': '시/도(학교 기준)', 'ARA2A': '도시규모(학교 기준)', 'ARA1B': '시/도(거주지 기준)', 'YTWIN': '쌍둥이 여부', 'YBRT1A': '생년', 'YBRT1B': '생월', 'YGENDER': '성별', 'YTIM1A01': '평일_기상(시)', 'YTIM1A02': '평일_기상(분)', 'YTIM1B01': '평일_취침(시)', 'YTIM1B02': '평일_취침(분)', 'YTIM1C01': '평일_수면의 질', 'YTIM1A03': '주말_기상(시)', 'YTIM1A04': '주말_기상(분)', 'YTIM1B03': '주말_취침(시)', 'YTIM1B04': '주말_취침(분)', 'YTIM1C02': '주말_수면의 질', 'YTIM1D01': '평일_부모님 대화시간', 'YTIM1D02': '주말_부모님 대화시간', 'YTIM1E01': '평일_학원 및 과외 시간', 'YTIM1E02': '주말_학원 및 과외 시간', 'YTIM1F01': '평일_인터넷 및 TV강의 시간', 'YTIM1F02': '주말_인터넷 및 TV강의 시간', 'YTIM1G01': '평일_방과 후 학교', 'YTIM1G02': '주말_방과 후 학교', 'YTIM1H01': '평일_스스로 공부하는 시간', 'YTIM1H02':

### **변수의 카테고리별 정보 딕셔너리**

In [ ]:
# codebook에서 카테고리에 따른 변수들을 딕셔너리 형태로 만드는 함수이다.
# 카테고리 = ['대영역', '중영역', '소영역', '조사항목', '하위요인']
# ex. ('수면시간', ['YTIM1A02', 'YTIM1B01', 'YTIM1B02', 'YTIM1A03', 'YTIM1A04', 'YTIM1B03', 'YTIM1B04', 'YTIM1A01', 'YTIM1A02', 'YTIM1B01', 'YTIM1B02', 'YTIM1A03', 'YTIM1A04', 'YTIM1B03', 'YTIM1B04'])

def get_category_info(category, codebooks):
    total_dict = {}

    keys = codebooks.keys()
    for key in keys:
        cat_dict = {}
        info = codebooks[key].loc[codebooks[key][category]!='-',:]
        for i in range(info.shape[0]):
            cat, var = info.iloc[i,:][[category,'변수명']]

            cat = re.sub('(^\[)|(\]$)','',cat)

            if cat in cat_dict.keys():
                cat_dict[cat].append(var)
            else:
                cat_dict[cat] = [var]
        total_dict[key] = cat_dict

    return total_dict

In [ ]:
# 변수의 카테고리별 정보 딕셔너리
bycat_info = {}
categories = ['대영역', '중영역', '소영역', '조사항목', '하위요인']

for category in categories:
    bycat_info[category] = get_category_info(category, codebooks)
    print(f'> {category}', end=' ')

# 저장
with open(f'./datasets/category_info.pkl', 'wb') as f:
    pickle.dump(bycat_info, f)

print(f'>>> 카테고리별 정보 저장 완료!')

> 대영역 > 중영역 > 소영역 > 조사항목 > 하위요인 >>> 카테고리별 정보 저장 완료!


In [ ]:
# 예시
i = 0
for x in bycat_info.items():
    print(x)
    i += 1
    if i == 3: break

('대영역', {'e4': {'배경변인': ['ARA1A', 'ARA2A', 'ARA1B', 'YTWIN', 'YBRT1A', 'YBRT1B', 'YGENDER'], '개인발달': ['YTIM1A01', 'YTIM1A02', 'YTIM1B01', 'YTIM1B02', 'YTIM1C01', 'YTIM1A03', 'YTIM1A04', 'YTIM1B03', 'YTIM1B04', 'YTIM1C02', 'YTIM1D01', 'YTIM1D02', 'YTIM1E01', 'YTIM1E02', 'YTIM1F01', 'YTIM1F02', 'YTIM1G01', 'YTIM1G02', 'YTIM1H01', 'YTIM1H02', 'YTIM1I01', 'YTIM1I02', 'YTIM1J01', 'YTIM1J02', 'YTIM1K01', 'YTIM1K02', 'YTIM1L01', 'YTIM1L02', 'YTIM1M01', 'YTIM1M02', 'YTIM1N01', 'YTIM1N02', 'YINT1A00', 'YINT1B00', 'YINT2A01', 'YINT2A02', 'YINT2A03', 'YINT2A04', 'YINT2A05', 'YINT2A06', 'YINT2A07', 'YINT2A08', 'YINT2A09', 'YINT2A10', 'YINT2A11', 'YINT2A12', 'YINT2A13', 'YINT2A14', 'YINT2A15', 'YINT2A16', 'YFUR2A01', 'YFUR2A02', 'YFUR2A03', 'YFUR2A04', 'YFUR2A05', 'YFUR2A06', 'YFUR2A07', 'YFUR2A08', 'YFUR2A08_T', 'YPSY1A01', 'YPSY1A02', 'YPSY1A03', 'YPSY1A04', 'YPSY1A05', 'YPSY2A01', 'YPSY2A02', 'YPSY2A03', 'YPSY2A04', 'YPSY3A01', 'YPSY3A02', 'YPSY3A03', 'YPSY3A04', 'YPSY3A05', 'YPSY3A06', 'YPSY3A0

### **범주형 변수의 값 설명 추출 딕셔너리**

In [ ]:
# codebook에서 변수가 범주형 자료일 때 값에 대한 설명을 딕셔너리 형태로 만드는 함수이다.
# ex. {'YONLINE', {0: 'TAPI 조사', 1: '온라인 조사'}

def get_value_info(file_path, sheet_name):
    value_dict = {}

    codebooks = pd.read_excel(file_path, sheet_name=sheet_name,skiprows=3,usecols=[4,5,6]).reset_index(drop=True)
    codebooks = codebooks.iloc[1:,:]
    codebooks = codebooks.loc[codebooks['값']!='-',:]

    codebooks = codebooks.replace(np.nan, '')

    for i in range(codebooks.shape[0]):
        var, val, desc = codebooks.iloc[i,:]

        if var != '':
            value_dict[var] = {val:desc}
            name = var
        else:
            value_dict[name].update({val:desc})

    return value_dict

In [ ]:
file_path = 'KCYPS2018 Codebook.xlsx'

sheets = ['CB_e', 'CB_m', 'CB_p']

value_dict = {}
cnt = 0
for sheet in sheets:
    if len(value_dict) == 0:
        value_dict = get_value_info(file_path, 'CB_e')
    else:
        temp = get_value_info(file_path, 'CB_m')
        for key, val in temp.items():
            if key in value_dict.keys():
                if value_dict[key] != value_dict[key]:
                    cnt += 1
                else:
                    continue
            else:
                value_dict.update({key:val})

if cnt == 0:
    print('>>> 변수는 같으나 정보가 다른 건은 없습니다.')

   # 저장
    with open(f'./datasets/value_info.pkl', 'wb') as f:
        pickle.dump(value_dict, f)

    print('>>> 변수의 값 설명 정보(통합) 저장 완료!')

>>> 변수는 같으나 정보가 다른 건은 없습니다.
>>> 변수의 값 설명 정보(통합) 저장 완료!


In [ ]:
# 예시
i = 0
for x in value_dict.items():
    print(x)
    i += 1
    if i == 3: break

('YONLINE', {0: 'TAPI 조사', 1: '온라인 조사'})
('SIBLING', {1: '형/오빠', 2: '누나/언니', 3: '남동생', 4: '여동생', 5: '기타'})
('SGRADE', {1: '초1', 2: '초2', 3: '초3', 4: '초4', 5: '초5', 6: '초6', 7: '중1', 8: '중2', 9: '중3', 10: '고1', 11: '고2', 12: '고3'})


## **3) 카테고리별 정보(공통 변수) 추출하기**

* 초등과 중등 데이터를 합치기 위해 공통된 변수를 찾고자 한다.

In [ ]:
common_var = list(set(variable_info['e4']) & set(variable_info['m1']))

# 변수의 카테고리별 정보 딕셔너리
bycat_common_info = {}
categories = ['대영역', '중영역', '소영역', '조사항목', '하위요인']

codebooks_common = {}

for k, v in codebooks.items():
    if k != 'p':
        is_include = v['변수명'].apply(lambda x: x in common_var)
        v = v.loc[is_include,:]

    codebooks_common[k] = v

for category in categories:
    bycat_common_info[category] = get_category_info(category, codebooks_common)
    print(f'> {category}', end=' ')

# 저장
with open(f'./datasets/category_common_info.pkl', 'wb') as f:
    pickle.dump(bycat_common_info, f)

print(f'>>> 카테고리별(공통변수) 정보 저장 완료!')

> 대영역 > 중영역 > 소영역 > 조사항목 > 하위요인 >>> 카테고리별(공통변수) 정보 저장 완료!


In [ ]:
# 예시
i = 0
for x in bycat_common_info.items():
    print(x)
    i += 1
    if i == 3: break

('대영역', {'e4': {'배경변인': ['ARA1A', 'ARA2A', 'ARA1B', 'YTWIN', 'YBRT1A', 'YBRT1B', 'YGENDER'], '개인발달': ['YTIM1A01', 'YTIM1A02', 'YTIM1B01', 'YTIM1B02', 'YTIM1C01', 'YTIM1A03', 'YTIM1A04', 'YTIM1B03', 'YTIM1B04', 'YTIM1C02', 'YTIM1D01', 'YTIM1D02', 'YTIM1E01', 'YTIM1E02', 'YTIM1F01', 'YTIM1F02', 'YTIM1G01', 'YTIM1G02', 'YTIM1H01', 'YTIM1H02', 'YTIM1I01', 'YTIM1I02', 'YTIM1J01', 'YTIM1J02', 'YTIM1K01', 'YTIM1K02', 'YTIM1L01', 'YTIM1L02', 'YTIM1M01', 'YTIM1M02', 'YTIM1N01', 'YTIM1N02', 'YINT1A00', 'YINT1B00', 'YINT2A01', 'YINT2A02', 'YINT2A03', 'YINT2A04', 'YINT2A05', 'YINT2A06', 'YINT2A07', 'YINT2A08', 'YINT2A09', 'YINT2A10', 'YINT2A11', 'YINT2A12', 'YINT2A13', 'YINT2A14', 'YINT2A15', 'YINT2A16', 'YFUR2A01', 'YFUR2A02', 'YFUR2A03', 'YFUR2A04', 'YFUR2A05', 'YFUR2A06', 'YFUR2A07', 'YFUR2A08', 'YFUR2A08_T', 'YPSY1A01', 'YPSY1A02', 'YPSY1A03', 'YPSY1A04', 'YPSY1A05', 'YPSY2A01', 'YPSY2A02', 'YPSY2A03', 'YPSY2A04', 'YPSY3A01', 'YPSY3A02', 'YPSY3A03', 'YPSY3A04', 'YPSY3A05', 'YPSY3A06', 'YPSY3A0

In [ ]:
# # 데이터 분리(대상별)
# data_e4Y = {1: data['e4Yw1'], 2: data['e4Yw2'], 3: data['e4Yw3'], 4: data['e4Yw4']}
# data_e4P = {1: data['e4Pw1'], 2: data['e4Pw2'], 3: data['e4Pw3'], 4: data['e4Pw4']}
# data_m1Y = {1: data['m1Yw1'], 2: data['m1Yw2'], 3: data['m1Yw3'], 4: data['m1Yw4']}
# data_m1P = {1: data['m1Pw1'], 2: data['m1Pw2'], 3: data['m1Pw3'], 4: data['m1Pw4']}

# by_subject_data = {'e4Y': data_e4Y, 'e4P': data_e4P, 'm1Y': data_m1Y, 'm1P': data_m1P}

# with open('./datasets/by_subject_data.pkl', 'wb') as f:
#     pickle.dump(by_subject_data, f)

In [ ]:
# # 데이터 분리(차수별)
# data_w1 = {'e4': data['e4Yw1'], 'e4p': data['e4Pw1'], 'm1': data['m1Yw1'], 'm1p': data['m1Pw1']}
# data_w2 = {'e4': data['e4Yw2'], 'e4p': data['e4Pw2'], 'm1': data['m1Yw2'], 'm1p': data['m1Pw2']}
# data_w3 = {'e4': data['e4Yw3'], 'e4p': data['e4Pw3'], 'm1': data['m1Yw3'], 'm1p': data['m1Pw3']}
# data_w4 = {'e4': data['e4Yw4'], 'e4p': data['e4Pw4'], 'm1': data['m1Yw4'], 'm1p': data['m1Pw4']}

# by_year_data = {'w1': data_w1, 'w2': data_w2, 'w3': data_w3, 'w4': data_w4}

# with open('./datasets/by_year_data.pkl', 'wb') as f:
#     pickle.dump(by_year_data, f)